In [1]:
!pip install opendatasets


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
dataset = 'https://www.kaggle.com/datasets/stackoverflow/pythonquestions'
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  hyeongchanim


Your Kaggle Key:

  ········


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


100% 558M/558M [00:40<00:00, 14.5MB/s] 

In [2]:
!pip install transformers

  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Using cached regex-2023.3.23-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (771 kB)
  Using cached filelock-3.10.7-py3-none-any.whl (10 kB)
  Using cached tokenizers-0.13.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)
  Using cached huggingface_hub-0.13.3-py3-none-any.whl (199 kB)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
data_dir = './pythonquestions'

os.listdir(data_dir)

['Questions.csv', 'Answers.csv', 'Tags.csv']

In [3]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


# 데이터 불러오기
df = pd.read_csv('./pythonquestions/Answers.csv', encoding='ISO-8859-1')

In [4]:
df.columns

Index(['Id', 'OwnerUserId', 'CreationDate', 'ParentId', 'Score', 'Body'], dtype='object')

In [5]:
df.head(5)

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,497,50.0,2008-08-02T16:56:53Z,469,4,<p>open up a terminal (Applications-&gt;Utilit...
1,518,153.0,2008-08-02T17:42:28Z,469,2,<p>I haven't been able to find anything that d...
2,536,161.0,2008-08-02T18:49:07Z,502,9,<p>You can use ImageMagick's convert utility f...
3,538,156.0,2008-08-02T18:56:56Z,535,23,<p>One possibility is Hudson. It's written in...
4,541,157.0,2008-08-02T19:06:40Z,535,20,"<p>We run <a href=""http://buildbot.net/trac"">B..."


In [6]:
df['Score']

0          4
1          2
2          9
3         23
4         20
          ..
987117     0
987118     2
987119     0
987120     0
987121     0
Name: Score, Length: 987122, dtype: int64

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
import opendatasets as od
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, random_split

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## 데이터 로드 및 전처리

In [ ]:
data = pd.read_csv('./pythonquestions/Answers.csv', encoding='ISO-8859-1')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

encoded_texts = tokenizer.batch_encode_plus(
    data.Body,
    add_special_tokens=True, 
    return_attention_mask=True,
    padding=True,
    truncation=True,
    max_length=256
)

input_ids = torch.tensor(encoded_texts['input_ids'])
attention_masks = torch.tensor(encoded_texts['attention_mask'])
labels = torch.tensor(data.label)

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


## 모델 생성

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

EPOCHS = 5

## 모델 학습

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    for batch in train_loader:
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
    model.eval()
    with torch.no_grad():
        val_loss = 0
        val_acc = 0
        for batch in val_loader:
            input_ids = batch[0].to(device)
            attention_masks = batch[1].to(device)
            labels = batch[2].to(device)
            
            outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
            val_loss += outputs.loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            val_acc += (preds == labels).sum().item()
        
        val_loss /= len(val_loader)
        val_acc /= len(val_loader)
        print(f'Epoch {epoch+1}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2%}')

In [ ]:
# 손실 그래프 출력
plt.plot(train_loss_list, label='Train')
plt.plot(val_loss_list, label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()